In [ ]:
# --- 1. 라이브러리, 경로, 로거 설정 ---

# 데이터 분석 및 처리를 위한 필수 라이브러리
import pandas as pd
import numpy as np
import random
import os, sys
from datetime import datetime                                         
from tqdm import tqdm
import warnings
import joblib
import json

# 시각화를 위한 라이브러리
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
import seaborn as sns
import shap
from sklearn.model_selection import learning_curve

# 모델링 및 기계 학습을 위한 라이브러리
import lightgbm as lgb
from catboost import CatBoostRegressor
from sklearn.model_selection import TimeSeriesSplit, train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.cluster import KMeans
from sklearn.metrics import mean_squared_error

# 하이퍼파라미터 최적화를 위한 라이브러리
import optuna 

# 불필요한 경고 메시지 무시
warnings.filterwarnings('ignore')
plt.style.use('ggplot')


# --- 1.1. 로거(실행 기록 로그 저장) 임포트 ---
# 사용자가 요청한 외부 logger.py 모듈을 임포합니다.
try:
    src_path = os.path.abspath(os.path.join(os.getcwd(), "../../src/log"))
    sys.path.insert(0, src_path)
    from logger import Logger
except ImportError:
    print("오류: 'logger.py'를 찾을 수 없습니다. 'src/log' 경로를 확인하세요.")
    # 간단한 대체 로거 정의
    class Logger:
        def __init__(self, log_path): print(f"대체 로거 활성화. 로그는 기록되지 않습니다.")
        def write(self, message, **kwargs): print(message)
        def start_redirect(self): pass
        def close(self): pass

# --- 1.2. 한글 폰트 설정 (나눔고딕) ---
try:
    font_path = '../../font/NanumFont/NanumGothic.ttf'
    if os.path.exists(font_path):
        fe = fm.FontEntry(fname=font_path, name='NanumGothic')
        fm.fontManager.ttflist.insert(0, fe)
        plt.rcParams.update({'font.size': 12, 'font.family': 'NanumGothic'})
    else:
        print("나눔고딕 폰트를 찾을 수 없어 기본 폰트로 설정됩니다.")
except Exception as e:
    print(f"폰트 설정 중 오류 발생: {e}")
    pass

# --- 1.3. 경로 및 환경 변수 설정 (사용자 요청 기반) ---
# 현재 시간 기준 년월일_시각 문자열 생성
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")

# 실행 로그 저장 경로 설정
LOG_DIR                         = '../../data/logs/price_prediction_9_logs'
LOG_FILENAME                    = f"price_prediction_9_{timestamp}.log"
LOG_PATH                        = os.path.join(LOG_DIR, LOG_FILENAME)
os.makedirs(LOG_DIR, exist_ok=True)
logger = Logger(log_path=LOG_PATH)

# 데이터 및 결과물 경로 설정
RAW_DIR                         = '../../data/processed/clean_data'
TRAIN_FILENAME                  = 'train.csv'
TEST_FILENAME                   = 'test.csv'
TRAIN_PATH                      = os.path.join(RAW_DIR, TRAIN_FILENAME)
TEST_PATH                       = os.path.join(RAW_DIR, TEST_FILENAME)

PARAMS_DIR                      = '../../data/processed/params'
PARAMS_FILENAME                 = 'best_params_9.json'
PARAMS_PATH                     = os.path.join(PARAMS_DIR, PARAMS_FILENAME)

SUBMISSION_DIR                  = '../../data/processed/submissions'
SUBMISSION_TEMPLATE_FILENAME    = 'baseline_code_sample_submission.csv'
SUBMISSION_FILENAME             = f'price_prediction_9_submission_{timestamp}.csv'
SUBMISSION_TEMPLATE_PATH        = os.path.join(SUBMISSION_DIR, SUBMISSION_TEMPLATE_FILENAME)
SUBMISSION_PATH                 = os.path.join(SUBMISSION_DIR, SUBMISSION_FILENAME)

IMAGE_DIR                       = '../../images/price_prediction_9/1'
IMAGE_FILENAME                  = 'price_prediction_9_model.pkl'
IMAGE_PATH                      = os.path.join(IMAGE_DIR, IMAGE_FILENAME)

MODEL_DIR                       = '../../model/price_prediction_9_{timestamp}'
MODEL_FILENAME                  = 'price_prediction_9_model.pkl'
MODEL_PATH                      = os.path.join(MODEL_DIR, MODEL_FILENAME)

# 결과 저장 디렉토리 생성
os.makedirs(PARAMS_DIR, exist_ok=True)
os.makedirs(SUBMISSION_DIR, exist_ok=True)
os.makedirs(IMAGE_DIR, exist_ok=True)
os.makedirs(MODEL_DIR, exist_ok=True)

logger.start_redirect()
logger.write("="*60)
logger.write(">> [price_prediction8] 아파트 가격 예측 모델링 시작")
logger.write("="*60)

In [ ]:
# ==============================================================================
# --- 2. 🚀 하이퍼파라미터 및 실행 환경 설정 (Config 클래스) ---
# ==============================================================================
class Config:
    IS_SAMPLING = False     # 샘플링 여부 (True: 샘플링, False: 전체 데이터 사용)
    SAMPLING_FRAC = 0.3     # 샘플링 비율 (0.3 = 30%)
    SEED = 42               # 랜덤 시드 고정
    N_SPLITS_TS = 10        # 시계열 교차 검증 분할 수
    N_TOP_FEATURES = 28     # 상위 28개 피처 사용
    N_TRIALS_OPTUNA = 30    # Optuna 하이퍼파라미터 최적화 시도 횟수

def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)

seed_everything(Config.SEED)
logger.write(">> [1단계 완료] 라이브러리, 경로, 로거 초기화 및 시드 고정 성공!")

In [ ]:
# ==============================================================================
# --- 3. 데이터 로드 및 병합 ---
# ==============================================================================
try:
    logger.write("\n>> [2단계 시작] 데이터 로드를 시작합니다.")
    train_df = pd.read_csv(TRAIN_PATH)
    test_df = pd.read_csv(TEST_PATH)
    submission_df = pd.read_csv(SUBMISSION_TEMPLATE_PATH)

    logger.write(f">> 원본 데이터 Shape - Train: {train_df.shape}, Test: {test_df.shape}")

    if Config.IS_SAMPLING:
        logger.write(f">> 샘플링 모드 활성화: 데이터의 {Config.SAMPLING_FRAC * 100}%만 사용합니다.")
        train_df = train_df.sample(frac=Config.SAMPLING_FRAC, random_state=Config.SEED).reset_index(drop=True)
        logger.write(f">> 샘플링 후 Train Shape: {train_df.shape}")
        
    logger.write(">> [2단계 완료] 데이터 로드 성공.")

except FileNotFoundError as e:
    logger.write(f">> [오류] 데이터 파일 로드 실패: {e}. '{TRAIN_PATH}' 또는 '{TEST_PATH}' 경로를 확인하세요.", print_error=True)


except Exception as e:
    logger.write(f">> [오류] 2단계(데이터 로드) 중 문제 발생: {e}", print_error=True)

In [ ]:
# ==============================================================================
# --- 셀 4: 피처 엔지니어링 (사용자 구조 복원 및 오류 수정) ---
# ==============================================================================
try:
    logger.write("\n>> [3단계 시작] 피처 엔지니어링을 시작합니다.")
    
    # target을 포함한 상태로 concat하여 통계 피처 생성 준비
    all_df = pd.concat([train_df, test_df], axis=0).reset_index(drop=True)
    logger.write(f">> Train/Test 병합 후 Shape: {all_df.shape}")

    # --- 4.1. 날짜 및 기본 파생 변수 ---
    try:
        # '계약년월'을 날짜 타입으로 변환 (YYYYMM 형식이라고 가정)
        all_df['계약년월'] = pd.to_datetime(all_df['계약년월'], format='%Y%m')
        # '계약년'과 '계약월' 피처 생성
        all_df['계약년'] = all_df['계약년월'].dt.year
        all_df['계약월'] = all_df['계약년월'].dt.month
        # '건물나이' 피처 생성
        all_df['건물나이'] = all_df['계약년'] - all_df['연식'] 
        # 월(Month) 피처를 주기성을 가지도록 sin, cos 변환
        all_df['계약월_sin'] = np.sin(2 * np.pi * all_df['계약월'] / 12)
        all_df['계약월_cos'] = np.cos(2 * np.pi * all_df['계약월'] / 12)
        logger.write(">> 4.1. 날짜/기본/주기성 피처 생성 완료.")
    except Exception as e:
        logger.write(f">> [오류] 4.1(날짜 피처) 생성 중 문제 발생: {e}", print_error=True)

    # --- 4.2. 교통 가중합 피처 ---
    try:
        # 가까울수록 높은 가중치를 부여한 교통 편의성 피처
        all_df['가중지하철'] = all_df['반경_1km_지하철역_수']*1.0 + all_df['반경_500m_지하철역_수']*1.5 + all_df['반경_300m_지하철역_수']*2.0
        all_df['가중버스'] = all_df['반경_1km_버스정류장_수']*1.0 + all_df['반경_500m_버스정류장_수']*1.5 + all_df['반경_300m_버스정류장_수']*2.0
        logger.write(">> 4.2. 교통 가중합 피처 생성 완료.")
    except Exception as e:
        logger.write(f">> [오류] 4.2(교통 피처) 생성 중 문제 발생: {e}", print_error=True)

    # --- 4.3. K-Means 군집화 피처 ---
    try:
        # 주요 수치형 변수들을 바탕으로 아파트 군집 생성
        cluster_features = ['좌표X', '좌표Y', '전용면적', '건물나이', '층']
        scaler = StandardScaler()
        df_scaled = scaler.fit_transform(all_df[cluster_features])
        kmeans = KMeans(n_clusters=10, random_state=Config.SEED, n_init=10)
        all_df['아파트군집'] = kmeans.fit_predict(df_scaled)
        logger.write(">> 4.3. K-Means 군집화 피처 생성 완료.")
    except Exception as e:
        logger.write(f">> [오류] 4.3(K-Means) 생성 중 문제 발생: {e}", print_error=True)
        
    # --- 4.4. 통계 피처 생성 ---
    try:
        # 데이터 유출을 방지하며, 지역/군집별 가격 통계 피처 생성
        train_only_df_mask = all_df['target'].notna()
        all_df.loc[train_only_df_mask, '면적당가격'] = np.log1p(all_df.loc[train_only_df_mask, 'target']) / all_df.loc[train_only_df_mask, '전용면적']
        
        dong_stats = all_df[train_only_df_mask].groupby('법정동').agg(동별_평균_면적당가격=('면적당가격', 'mean'), 동별_std_면적당가격=('면적당가격', 'std')).reset_index()
        gu_stats = all_df[train_only_df_mask].groupby('자치구').agg(구별_평균_면적당가격=('면적당가격', 'mean'), 구별_std_면적당가격=('면적당가격', 'std')).reset_index()
        cluster_stats = all_df[train_only_df_mask].groupby('아파트군집').agg(군집별_평균_면적당가격=('면적당가격', 'mean'), 군집별_std_면적당가격=('면적당가격', 'std')).reset_index()
        
        all_df = pd.merge(all_df, dong_stats, on='법정동', how='left')
        all_df = pd.merge(all_df, gu_stats, on='자치구', how='left')
        all_df = pd.merge(all_df, cluster_stats, on='아파트군집', how='left')
        all_df = all_df.drop(columns=['면적당가격'])
        logger.write(">> 4.4. 법정동/자치구/아파트군집 기반 통계 피처 생성 완료.")
    except Exception as e:
        logger.write(f">> [오류] 4.4(통계 피처) 생성 중 문제 발생: {e}", print_error=True)
        
    # --- 4.5. 상호작용 피처 ---
    try:
        # 변수 간의 상호작용을 고려한 파생변수 생성
        all_df['면적_x_나이'] = all_df['전용면적'] * all_df['건물나이']
        all_df['면적_x_층'] = all_df['전용면적'] * all_df['층']
        all_df['강남_x_면적'] = all_df['강남3구여부'] * all_df['전용면적']
        logger.write(">> 4.5. 상호작용 피처 생성 완료.")
    except Exception as e:
        logger.write(f">> [오류] 4.5(상호작용 피처) 생성 중 문제 발생: {e}", print_error=True)

    # --- 4.6. 최종 처리 ---
    try:
        # 모델 학습에 사용하지 않을 원본 컬럼들을 명시적으로 제거
        cols_to_drop = ['계약년월', '계약일자']
        all_df = all_df.drop(columns=cols_to_drop)
        
        # 결측치 처리 후 데이터 분리
        logger.write(f">> 결측치 처리 전, NA 개수: {all_df.isna().sum().sum()}")
        all_df = all_df.fillna(0) # 시계열 피처에서 발생한 NA를 0으로 채움
        logger.write(f">> 결측치 처리 후, NA 개수: {all_df.isna().sum().sum()}")
        
        # 데이터를 훈련/테스트용으로 분리하고, 타겟 변수(y) 생성
        train_processed = all_df[all_df['target'] != 0].copy()
        X_train = train_processed.drop(columns=['target'])
        y_train_log = np.log1p(train_processed['target'])
        X_test = all_df[all_df['target'] == 0].drop(columns=['target']).copy()
        
        logger.write(f">> 최종 피처 수: {len(X_train.columns)}")
        logger.write(">> 4.6. 최종 데이터 분리 및 전처리 완료.")
    except Exception as e:
        logger.write(f">> [오류] 4.6(최종 처리) 중 문제 발생: {e}", print_error=True)

    logger.write(">> [3단계 완료] 피처 엔지니어링 성공.")
    
except Exception as e:
    logger.write(f">> [오류] 3단계(피처 엔지니어링) 전체 프로세스 중 문제 발생: {e}", print_error=True)

In [ ]:
# ==============================================================================
# --- 5. ⚡️ 빠른 피처 선택 ---
# ==============================================================================

try:
    logger.write(f"\n>> [4단계 시작] 상위 {Config.N_TOP_FEATURES}개 피처를 선택합니다.")
    
    # 피처 선택을 위한 임시 데이터프레임 복사
    X_train_fs = X_train.copy()
    
    # 범주형 피처들의 타입을 LightGBM이 인식할 수 있는 'category'로 변환
    categorical_fs = ['자치구', '법정동', '브랜드등급', '아파트군집']
    for col in categorical_fs:
        if col in X_train_fs.columns:
            X_train_fs[col] = X_train_fs[col].astype('category')

    # 임시 모델로 피처 중요도 계산
    temp_model = lgb.LGBMRegressor(device='cuda', random_state=Config.SEED)
    temp_model.fit(X_train_fs, y_train_log)
    
    feature_importances = pd.DataFrame({'feature': X_train.columns, 'importance': temp_model.feature_importances_})
    feature_importances = feature_importances.sort_values('importance', ascending=False)
    logger.write(">> 피처 중요도 :")
    logger.write(str(feature_importances))
    
    # 전체 피처 목록
    all_features = X_train.columns.tolist()
    
    # 중요도 상위 피처만 선택
    top_features = feature_importances['feature'].head(Config.N_TOP_FEATURES).tolist()
    X_train = X_train[top_features]
    X_test = X_test[top_features]
    logger.write(f">> 선택된 상위 피쳐 수: {len(top_features)}")
    logger.write(f">> 선택된 상위 피쳐: {top_features}")
    
    # 피처 선택 후, 사용하지 않는 피처 목록
    discarded_features = [f for f in all_features if f not in top_features]
    logger.write(f">> 사용하지 않는 피처 수: {len(discarded_features)}")
    logger.write(f">> 사용하지 않는 피처: {discarded_features}")

    logger.write(f">> 최종 피쳐 개수: 총 {len(all_features)}개 중 {len(top_features)}개 선택, {len(discarded_features)}개 제외.")
    logger.write(">> [4단계 완료] 피처 선택 성공.")
    
except Exception as e:
    logger.write(f">> [오류] 4단계(피처 선택) 중 문제 발생: {e}", print_error=True)

In [ ]:
# ==============================================================================
# --- 6. LightGBM 모델 학습 ---
# ==============================================================================
try:
    logger.write(f"\n>> [5단계 시작] LightGBM 모델 학습을 시작합니다. (CV 폴드 수: {Config.N_SPLITS_TS})")
    
    # 사용자 정의 최적의 파라미터
    best_params = { 
        'learning_rate': 0.04, 
        'feature_fraction': 0.9, 
        'bagging_fraction': 0.85, 
        'bagging_freq': 3, 
        'num_leaves': 100, 
        'max_depth': 20, 
        'min_child_samples': 35 }
    logger.write(f">> 사용자가 정의한 최적 파라미터를 사용합니다: {best_params}")
    
    # 최종 학습을 위한 파라미터 미세 조정
    final_params = best_params.copy()
    final_params['learning_rate'] *= 0.8
    final_params['n_estimators'] = 3000
    final_params.update({
        'device': 'cuda', 
        'objective': 'regression_l1', 
        'metric': 'rmse', 
        'verbose': -1, 
        'n_jobs': -1, 
        'seed': Config.SEED})
    logger.write(f">> 최종 LGBM 학습 파라미터: {final_params}")
    
    # LGBM 학습용 데이터 준비
    X_train_lgbm, X_test_lgbm = X_train.copy(), X_test.copy()
    categorical_cols_lgbm = ['자치구', '법정동', '브랜드등급', '아파트군집']
    
    logger.write(">> LightGBM 학습을 위해 범주형 피처 타입을 'category'로 변환합니다.")
    logger.write(f">> 대상 컬럼: {categorical_cols_lgbm}")
    
    for col in categorical_cols_lgbm:
        if col in X_train_lgbm.columns:
            X_train_lgbm[col] = X_train_lgbm[col].astype('category')
            X_test_lgbm[col] = X_test_lgbm[col].astype('category')
    logger.write(">> 'category' 타입 변환 완료.")
            
    # TimeSeriesSplit 교차 검증
    ts_cv = TimeSeriesSplit(n_splits=Config.N_SPLITS_TS)
    lgbm_fold_models = []
    lgbm_oof_preds = np.zeros(len(X_train_lgbm))
    lgbm_test_preds = np.zeros(len(X_test_lgbm))
    lgbm_fold_scores = []

    for fold, (train_idx, val_idx) in enumerate(ts_cv.split(X_train_lgbm)):
        logger.write(f"--- LightGBM Fold {fold+1}/{Config.N_SPLITS_TS} 학습 시작 ---")
        X_train_fold, y_train_fold = X_train_lgbm.iloc[train_idx], y_train_log.iloc[train_idx]
        X_val_fold, y_val_fold = X_train_lgbm.iloc[val_idx], y_train_log.iloc[val_idx]
        
        # 데이터 분할 정보 로깅
        logger.write(f"- Train Index: {train_idx[0]} ~ {train_idx[-1]} (size: {len(train_idx)})")
        logger.write(f"- Validation Index: {val_idx[0]} ~ {val_idx[-1]} (size: {len(val_idx)})")
        
        # 모델 학습
        model = lgb.LGBMRegressor(**final_params)
        model.fit(X_train_fold, y_train_fold, 
                  eval_set=[(X_val_fold, y_val_fold)], 
                  eval_metric='rmse', 
                  callbacks=[lgb.early_stopping(200, verbose=False)])
        
        # OOF 및 테스트 데이터 예측
        val_preds = model.predict(X_val_fold)
        lgbm_oof_preds[val_idx] = model.predict(X_val_fold)
        lgbm_test_preds += model.predict(X_test_lgbm) / Config.N_SPLITS_TS
        lgbm_fold_models.append(model)
        
        # 해당 Fold의 결과 로깅
        fold_rmse = np.sqrt(mean_squared_error(y_val_fold, val_preds))
        lgbm_fold_scores.append(fold_rmse)
        logger.write(f"- Fold {fold+1} RMSE: {fold_rmse:.5f}")
        logger.write(f"- Best Iteration: {model.best_iteration_}")
    
    # CV 학습 결과 요약 로그
    oof_rmse = np.sqrt(mean_squared_error(y_train_log, lgbm_oof_preds))
    logger.write("\n>> LightGBM CV 학습 결과 요약:")
    logger.write(f"- 각 Fold별 RMSE: {[round(score, 5) for score in lgbm_fold_scores]}")
    logger.write(f"- 평균 Fold RMSE: {np.mean(lgbm_fold_scores):.5f} (±{np.std(lgbm_fold_scores):.5f})")
    logger.write(f"- 전체 OOF RMSE: {oof_rmse:.5f}")
    logger.write(">> [5단계 완료] LightGBM 모델 학습 성공.")
    
except Exception as e:
    logger.write(f">> [오류] 5단계(LightGBM 학습) 중 문제 발생: {e}", print_error=True)

In [ ]:
# ==============================================================================
# --- 7. CatBoost 모델 학습 ---
# ==============================================================================

try:
    logger.write(f"\n>> [6단계 시작] CatBoost 모델 학습을 시작합니다. (CV 폴드 수: {Config.N_SPLITS_TS})")
    
    # CatBoost가 처리할 범주형 피처 목록 정의
    categorical_cols_cat = ['자치구', '법정동', '브랜드등급', '아파트군집']
    existing_categorical_cols = [col for col in categorical_cols_cat if col in X_train.columns]
    logger.write(f">> CatBoost 범주형 피처: {existing_categorical_cols}")

    ts_cv = TimeSeriesSplit(n_splits=Config.N_SPLITS_TS)
    cat_fold_models, cat_oof_preds = [], np.zeros(len(X_train))
    cat_test_preds = np.zeros(len(X_test))
    cat_fold_scores = []
    
    # CatBoost 학습 파라미터
    cat_params = {
        'iterations': 3000, 
        'learning_rate': 0.05, 
        'depth': 8, 
        'l2_leaf_reg': 3, 
        'loss_function': 'RMSE', 
        'eval_metric': 'RMSE', 
        'random_seed': Config.SEED, 
        'verbose': 0, 
        'task_type': 'GPU'}
    logger.write(f">> CatBoost 학습 파라미터: {cat_params}")

    for fold, (train_idx, val_idx) in enumerate(ts_cv.split(X_train)):
        logger.write(f"--- CatBoost Fold {fold+1}/{Config.N_SPLITS_TS} 학습 시작 ---")
        
        X_train_fold, y_train_fold = X_train.iloc[train_idx], y_train_log.iloc[train_idx]
        X_val_fold, y_val_fold = X_train.iloc[val_idx], y_train_log.iloc[val_idx]
        
        # 데이터 분할 정보 로깅
        logger.write(f"- Train Index: {train_idx[0]} ~ {train_idx[-1]} (size: {len(train_idx)})")
        logger.write(f"- Validation Index: {val_idx[0]} ~ {val_idx[-1]} (size: {len(val_idx)})")
        
        # 모델 학습
        cat_model = CatBoostRegressor(**cat_params)
        cat_model.fit(X_train_fold, y_train_fold, 
                      eval_set=[(X_val_fold, y_val_fold)], 
                      early_stopping_rounds=200, 
                      cat_features=existing_categorical_cols, 
                      verbose=0)
        
        # OOF 및 테스트 데이터 예측
        val_preds = cat_model.predict(X_val_fold)
        cat_oof_preds[val_idx] = val_preds
        cat_test_preds += cat_model.predict(X_test) / Config.N_SPLITS_TS
        cat_fold_models.append(cat_model)
        
        # 해당 Fold의 결과 로깅
        fold_rmse = np.sqrt(mean_squared_error(y_val_fold, val_preds))
        cat_fold_scores.append(fold_rmse)
        logger.write(f"- Fold {fold+1} RMSE: {fold_rmse:.5f}")
        logger.write(f"- Best Iteration: {cat_model.get_best_iteration()}")
    
    # CV 학습 결과 요약 로그
    oof_rmse = np.sqrt(mean_squared_error(y_train_log, cat_oof_preds))
    logger.write("\n>> CatBoost CV 학습 결과 요약:")
    logger.write(f"- 각 Fold별 RMSE: {[round(score, 5) for score in cat_fold_scores]}")
    logger.write(f"- 평균 Fold RMSE: {np.mean(cat_fold_scores):.5f} (±{np.std(cat_fold_scores):.5f})")
    logger.write(f"- 전체 OOF RMSE: {oof_rmse:.5f}")
    logger.write(">> [6단계 완료] CatBoost 모델 학습 성공.")
    
except Exception as e:
    logger.write(f">> [오류] 6단계(CatBoost 학습) 중 문제 발생: {e}", print_error=True)

In [ ]:
# ==============================================================================
# --- 8. 🚀 최종 예측 결합 및 제출 ---
# ==============================================================================

try:
    logger.write("\n>> [7단계 시작] 최종 예측 결합 및 제출을 시작합니다.")
    
    # 각 모델의 OOF(Out-of-Fold) 검증 점수 계산
    lgbm_oof_rmse = np.sqrt(mean_squared_error(y_train_log, lgbm_oof_preds))
    cat_oof_rmse = np.sqrt(mean_squared_error(y_train_log, cat_oof_preds))
    logger.write(f">> LightGBM OOF RMSE: {lgbm_oof_rmse:.5f}")
    logger.write(f">> CatBoost OOF RMSE: {cat_oof_rmse:.5f}")

    # 앙상블 가중치 설정 및 예측 결합
    lgbm_weight, cat_weight = 0.6, 0.4
    ensembled_preds = lgbm_test_preds * lgbm_weight + cat_test_preds * cat_weight
    logger.write(f">> 적용된 앙상블 가중치 - LGBM: {lgbm_weight}, CatBoost: {cat_weight}")
    
    # 앙상블된 예측값(로그 스케일)의 통계 정보 로깅
    logger.write(f">> 앙상블 예측값(로그 스케일) 요약: Min({np.min(ensembled_preds):.4f}), Max({np.max(ensembled_preds):.4f}), Mean({np.mean(ensembled_preds):.4f})")
    
    # 최종 예측값 스케일 복원
    final_predictions = np.expm1(ensembled_preds)
    
    # 음수 값 처리 전, 후 정보 로깅
    negative_preds_count = (final_predictions < 0).sum()
    logger.write(f">> 음수로 예측된 값의 수: {negative_preds_count}개")
    final_predictions[final_predictions < 0] = 0
    
    # 최종 예측값(원래 스케일)의 통계 정보 로깅
    logger.write(f">> 최종 예측값(원래 스케일) 요약: Min({np.min(final_predictions):.2f}), Max({np.max(final_predictions):.2f}), Mean({np.mean(final_predictions):.2f})")

    # 제출 파일 생성
    submission_df['target'] = final_predictions.astype(int)
    submission_df.to_csv(SUBMISSION_PATH, index=False)
    logger.write(f">> 제출 파일 '{SUBMISSION_PATH}' 생성이 완료되었습니다.")
    # 제출 파일 형태 로깅
    logger.write(f">> 최종 제출 파일 Shape: {submission_df.shape}")
    
    # 학습된 모든 모델 저장
    for i, model in enumerate(lgbm_fold_models):
        joblib.dump(model, os.path.join(MODEL_DIR, f'lgbm_fold{i+1}.pkl'))
    for i, model in enumerate(cat_fold_models):
        joblib.dump(model, os.path.join(MODEL_DIR, f'cat_fold{i+1}.pkl'))
    logger.write(">> 모든 Fold 모델 저장이 완료되었습니다.")
    
    logger.write(">> [7단계 완료] 제출 파일 및 모델 저장 성공.")
    
except Exception as e:
    logger.write(f">> [오류] 7단계(제출 및 저장) 중 문제 발생: {e}", print_error=True)

In [ ]:
# ==============================================================================
# --- 9. 📊 최종 모델 결과 시각화, 분석 및 이미지 저장 (최종 수정) ---
# ==============================================================================

# --- 1.2. 한글 폰트 설정 (나눔고딕) ---
import matplotlib.font_manager as fm
try:
    font_path = '../../font/NanumFont/NanumGothic.ttf'
    if os.path.exists(font_path):
        fe = fm.FontEntry(fname=font_path, name='NanumGothic')
        fm.fontManager.ttflist.insert(0, fe)
        plt.rcParams.update({'font.size': 12, 'font.family': 'NanumGothic'})
    else:
        print("나눔고딕 폰트를 찾을 수 없어 기본 폰트로 설정됩니다.")
except Exception as e:
    print(f"폰트 설정 중 오류 발생: {e}")
    pass

try:
    logger.write("\n>> [8단계 시작] 모델 결과 시각화 및 분석 파일 저장을 시작합니다...")
    logger.write(f">> 시각화 결과가 저장될 경로: {IMAGE_DIR}")

    # [정리] 시각화에 필요한 변수들을 이 단계 시작 부분에서 명확하게 정의합니다.
    
    # 1. 앙상블 OOF(Out-of-Fold) 예측값 및 관련 변수 생성
    ensembled_oof_preds = lgbm_oof_preds * lgbm_weight + cat_oof_preds * cat_weight
    residuals = np.expm1(y_train_log) - np.expm1(ensembled_oof_preds)
    
    # 2. LightGBM 모델 기반 피처 중요도 계산
    all_importances = pd.DataFrame()
    for i, model in enumerate(lgbm_fold_models):
        fold_importance = pd.DataFrame({'feature': X_train.columns, 'importance': model.feature_importances_, 'fold': i + 1})
        all_importances = pd.concat([all_importances, fold_importance], axis=0)
    mean_importances = all_importances.groupby('feature')['importance'].mean().sort_values(ascending=False)

    # 3. SHAP 분석을 위한 변수 준비 (LGBM 마지막 모델 기준)
    explainer = shap.TreeExplainer(lgbm_fold_models[-1])
    shap_sample = X_train_lgbm.sample(2000, random_state=Config.SEED) if len(X_train_lgbm) > 2000 else X_train_lgbm
    shap_values = explainer.shap_values(shap_sample)

    # --- 이제부터 순서대로 시각화를 진행합니다. ---

    # 1. 실제 값 vs OOF 예측 값 비교 (앙상블 기준)
    try:
        logger.write(">> 8.1. 실제 값 vs OOF 예측 값 비교 이미지 저장 중...")
        plt.figure(figsize=(10, 10)); sns.scatterplot(x=np.expm1(y_train_log), y=np.expm1(ensembled_oof_preds), alpha=0.3)
        plt.plot([0, np.expm1(y_train_log).max()], [0, np.expm1(y_train_log).max()], 'r--', lw=2)
        plt.xlabel("실제 값 (원)"); plt.ylabel("앙상블 OOF 예측 값 (원)"); plt.title('실제 값 vs OOF 예측 값 비교 (앙상블)', fontsize=16)
        plt.savefig(os.path.join(IMAGE_DIR, '01_Actual_vs_OOF_Scatter.png'), bbox_inches='tight'); plt.close()
        logger.write(">> 8.1. 실제 값 vs OOF 예측 값 비교 이미지 저장 완료.")
    except Exception as e:
        logger.write(f">> [오류] 8.1(실제값vs예측값) 시각화 중 문제 발생: {e}", print_error=True)

    # 2. 잔차 분포 (앙상블 기준)
    try:
        logger.write(">> 8.2. 잔차 분포 확인 이미지 저장 중...")
        plt.figure(figsize=(10, 6)); sns.histplot(residuals, kde=True, bins=50)
        plt.title('잔차(실제-예측) 분포 (앙상블 OOF 기반)', fontsize=16); plt.xlabel("잔차 (원)")
        plt.savefig(os.path.join(IMAGE_DIR, '02_Residuals_Distribution.png'), bbox_inches='tight'); plt.close()
        logger.write(">> 8.2. 잔차 분포 확인 이미지 저장 완료.")
    except Exception as e:
        logger.write(f">> [오류] 8.2(잔차 분포) 시각화 중 문제 발생: {e}", print_error=True)

    # 3. 잔차 vs 예측값 플롯
    try:
        logger.write(">> 8.3. 잔차 vs 예측값 플롯 이미지 저장 중...")
        plt.figure(figsize=(10, 6)); sns.scatterplot(x=np.expm1(ensembled_oof_preds), y=residuals, alpha=0.3)
        plt.axhline(y=0, color='r', linestyle='--')
        plt.xlabel("앙상블 OOF 예측 값 (원)"); plt.ylabel("잔차 (원)"); plt.title('잔차 vs 예측값 플롯 (앙상블 OOF 기반)', fontsize=16)
        plt.savefig(os.path.join(IMAGE_DIR, '03_Residuals_vs_Predicted.png'), bbox_inches='tight'); plt.close()
        logger.write(">> 8.3. 잔차 vs 예측값 플롯 이미지 저장 완료.")
    except Exception as e:
        logger.write(f">> [오류] 8.3(잔차vs예측값) 시각화 중 문제 발생: {e}", print_error=True)
        
    # 4. 예측 분포 비교
    try:
        logger.write(">> 8.4. OOF 예측과 테스트 예측의 분포 비교 이미지 저장 중...")
        plt.figure(figsize=(10, 6)); sns.kdeplot(np.expm1(ensembled_oof_preds), label='앙상블 OOF 예측 값', fill=True)
        sns.kdeplot(final_predictions, label='최종 테스트 예측 값', fill=True)
        plt.title('OOF 예측과 테스트 예측의 분포 비교', fontsize=16); plt.xlabel("예측 값 (원)"); plt.legend()
        plt.savefig(os.path.join(IMAGE_DIR, '04_Prediction_Distribution_Comparison.png'), bbox_inches='tight'); plt.close()
        logger.write(">> 8.4. OOF 예측과 테스트 예측의 분포 비교 이미지 저장 완료.")
    except Exception as e:
        logger.write(f">> [오류] 8.4(예측 분포) 시각화 중 문제 발생: {e}", print_error=True)

    # 5. 피처 중요도 (LGBM 기준)
    try:
        logger.write(">> 8.5. 피처 중요도 이미지 저장 중...")
        plt.figure(figsize=(12, 10)); sns.barplot(x=mean_importances.head(20).values, y=mean_importances.head(20).index)
        plt.title('상위 20개 피처 중요도 (LGBM 평균)', fontsize=16)
        plt.savefig(os.path.join(IMAGE_DIR, '05_Feature_Importance.png'), bbox_inches='tight'); plt.close()
        logger.write(">> 8.5. 피처 중요도 이미지 저장 완료.")
    except Exception as e:
        logger.write(f">> [오류] 8.5(피처 중요도) 시각화 중 문제 발생: {e}", print_error=True)

    # 6. SHAP 요약 플롯 (LGBM 기준)
    try:
        logger.write(">> 8.6. SHAP 요약 플롯 분석 및 저장 중...")
        plt.figure()
        shap.summary_plot(shap_values, shap_sample, show=False)
        plt.title("SHAP 요약 플롯 (LGBM 마지막 폴드 모델)", fontsize=16)
        plt.savefig(os.path.join(IMAGE_DIR, '06_SHAP_Summary_Plot.png'), bbox_inches='tight'); plt.close()
        logger.write(">> 8.6. SHAP 요약 플롯 이미지 저장 완료.")
    except Exception as e:
        logger.write(f">> [오류] 8.6(SHAP 요약) 분석 중 문제 발생: {e}", print_error=True)
        
    # # 7. SHAP 의존성 플롯
    try:
        logger.write(">> 8.7. SHAP 의존성 플롯 저장 중...")
        top_3_features = mean_importances.head(3).index
        
        for feature in top_3_features:
            # '법정동' 피처일 경우에만 필터링 로직을 적용합니다.
            if feature == '법정동':
                # 1. '법정동' 피처가 shap_sample 데이터프레임에서 몇 번째 열인지 인덱스를 찾습니다.
                feature_idx = list(shap_sample.columns).index(feature)
                
                # 2. 각 법정동 코드별로 평균 SHAP 값의 절댓값을 계산합니다.
                #    (어떤 동이 가격에 가장 큰 영향을 주는지 파악하기 위함)
                bjdong_shap_means = {}
                unique_bjdongs = shap_sample[feature].unique()
                for bjdong_code in unique_bjdongs:
                    mask = (shap_sample[feature] == bjdong_code)
                    mean_shap_value = np.abs(shap_values[mask, feature_idx]).mean()
                    bjdong_shap_means[bjdong_code] = mean_shap_value
                
                # 3. 계산된 SHAP 값이 높은 순서대로 상위 20개의 법정동을 선택합니다.
                #    [:20] 이 숫자를 바꾸면 개수를 조절할 수 있습니다. (예: :30)
                top_20_bjdongs = sorted(bjdong_shap_means, key=bjdong_shap_means.get, reverse=True)[:20]
                
                # 4. 원본 샘플 데이터에서 상위 20개 법정동에 해당하는 데이터만 필터링합니다.
                top_20_mask = shap_sample[feature].isin(top_20_bjdongs)
                
                # 5. 필터링된 데이터(shap_values와 shap_sample)로 의존성 플롯을 생성합니다.
                shap.dependence_plot(
                    ind=feature, 
                    shap_values=shap_values[top_20_mask], 
                    features=shap_sample[top_20_mask], 
                    interaction_index="auto", 
                    show=False
                )
                
                # 6. 플롯을 보기 좋게 꾸미고 저장합니다.
                plt.title(f"SHAP Dependence Plot for '{feature}' (Top 20)", fontsize=16)
                plt.xticks(rotation=45, ha='right') # 라벨이 길 수 있으니 45도 회전
                plt.tight_layout()
                plt.savefig(os.path.join(IMAGE_DIR, f'07_SHAP_Dependence_{feature}_Top20.png'), bbox_inches='tight')
                plt.close()

            else: # '법정동'이 아닌 다른 피처(건물나이 등)는 기존 방식대로 전체 플롯을 저장합니다.
                shap.dependence_plot(feature, shap_values, shap_sample, interaction_index="auto", show=False)
                plt.title(f"SHAP Dependence Plot for '{feature}'", fontsize=16)
                plt.savefig(os.path.join(IMAGE_DIR, f'07_SHAP_Dependence_{feature}.png'), bbox_inches='tight')
                plt.close()
                
        logger.write(">> 8.7. SHAP 의존성 플롯 이미지 저장 완료.")
    except Exception as e:
        logger.write(f">> [오류] 8.7(SHAP 의존성) 시각화 중 문제 발생: {e}", print_error=True)
        
    # 8. 학습 곡선
    try:
        logger.write(">> 8.8. 학습 곡선 이미지 저장 중...")

        # LGBM이 학습했던 'category' 타입의 데이터를 사용합니다.
        train_sizes, train_scores, validation_scores = learning_curve(
            estimator=lgb.LGBMRegressor(**best_params, verbosity=-1, random_state=Config.SEED, device='cuda'),
            X=X_train_lgbm, # X_train -> X_train_lgbm 으로 수정
            y=y_train_log, 
            train_sizes=np.linspace(0.1, 1.0, 5),
            cv=TimeSeriesSplit(n_splits=3), 
            scoring='neg_root_mean_squared_error', 
            n_jobs=-1)
        
        train_scores_mean = -train_scores.mean(axis=1)
        validation_scores_mean = -validation_scores.mean(axis=1)

        plt.figure(figsize=(10, 6))
        plt.plot(train_sizes, train_scores_mean, 'o-', color="r", label="Training score")
        plt.plot(train_sizes, validation_scores_mean, 'o-', color="g", label="Cross-validation score")
        plt.title("학습 곡선 (Learning Curve)", fontsize=16)
        plt.xlabel("학습 데이터 샘플 수")
        plt.ylabel("RMSE")
        plt.legend(loc="best")
        plt.grid()
        plt.savefig(os.path.join(IMAGE_DIR, '08_learning_curve.png'), bbox_inches='tight')
        plt.close()
        logger.write(">> 8.8. 학습 곡선 이미지 저장 완료.")
    except Exception as e:
        logger.write(f">> [오류] 8.8(학습 곡선) 생성 중 문제 발생: {e}", print_error=True)

    logger.write(">> [8단계 완료] 모든 시각화 및 분석 파일 저장 완료.")

except Exception as e:
    logger.write(f">> [오류] 8단계(시각화 및 분석) 전체 프로세스 중 문제 발생: {e}", print_error=True)
finally:
    logger.write("\n" + "="*60)
    logger.write("🎉 모든 프로세스가 성공적으로 종료되었습니다.")
    logger.write("="*60)
    # logger.close()